In [1]:
import json
import onnx
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()

    def forward(self, u, U, X):
        U1 = torch.concat((U[1:, :], u))
        x = torch.stack([U1[-3, 0], U1[-2, 1], U1[-1, 2]]).unsqueeze(0)
        X1 = torch.concat((X[1:, :], x))
        return x, X1, U1

In [3]:
FEATURES = 5
TARGETS = 3
T = 10
# Create three tensors
u = torch.ones((1, FEATURES))
U = torch.ones((T - 1, FEATURES)) * torch.arange(T, 1, -1).unsqueeze(1)
X = torch.ones(T, TARGETS)

# Create the model
model = ExampleModel()

# Run the model
output = model(u, U, X)
print([i.shape for i in output])

[torch.Size([1, 3]), torch.Size([10, 3]), torch.Size([9, 5])]


In [4]:
model_name = "example4"
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (u, U, X),
    f"{model_name}.onnx",
    verbose=False,
    input_names=["u", "U", "X"],
    output_names=["x", "X1", "U1"],
)

# Load the model
onnx_model = onnx.load(f"{model_name}.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example to test FMU with local variables."

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, f"{model_name}.onnx")


## Generating model description

Create and save the model description to be provided to ONNX2FMU.

In [5]:
model_description = {
    "name": "example4",
    "description": "Example to test FMU with local variables.",
    "FMIVersion": "2.0",
    "inputs": [
        {
            "name": "u",
            "description": "A vector of control variables at time t."
        },
    ],
    "outputs": [
        {
            "name": "x",
            "description": "The state of the system at time t+1."
        }
    ],
    "locals": [
        {
            "nameIn": "X",
            "nameOut": "X1",
            "description": "The history of states from t-N to t."
        },
        {
            "nameIn": "U",
            "nameOut": "U1",
            "description": "The history of control variables frmo t-N to t-1."
        }
    ]
}

# Save model description
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)

## Generating input file and output for testing

In [6]:
time_steps = 100
U_hist = np.ones((time_steps, FEATURES)) * np.arange(time_steps)[:, None]
columns = [f"u_0_{i}" for i in range(FEATURES)]
index = pd.Index(data=np.arange(time_steps), name='time')
df = pd.DataFrame(
    data=U_hist,
    columns=columns,
    index=index
).to_csv(f"input.csv")

results = torch.empty((time_steps, TARGETS))
U = torch.zeros((T - 1, FEATURES))
X = torch.zeros((T, TARGETS))
for i, u in enumerate(U_hist):
    x, X1, U1 = model(
        torch.tensor(u).unsqueeze(0), U, X
    )
    U, X = U1, X1
    results[i] = x

output = pd.DataFrame(
    data=results.detach().numpy(),
    columns=[f"x_0_{i}" for i in range(results.shape[1])],
    index=index
)

output.to_csv("output.csv")